Workbook

First step is to load in the data into separate pandas dataframes

In [1]:
import numpy as np
import pandas as pd

total_gdp = pd.read_csv("total_gdp_us_inflation_adjusted.csv")
total_gdp = total_gdp.set_index('country')

gdp_percap = pd.read_csv("gdppercapita_us_inflation_adjusted.csv")
gdp_percap = gdp_percap.set_index('country')
gini = pd.read_csv("gini.csv")
gini = gini.set_index('country')

Convert values in each df from string to float, parsing special characters as needed. E.g., '400B' is 400.0e9.

In [2]:
total_gdp_copy = total_gdp.copy()
gdp_percap_copy = gdp_percap.copy()
    

for country in total_gdp_copy.index:
    for year in total_gdp_copy:
        val = total_gdp_copy.loc[country][year]
        if isinstance(val, float):
            continue    
        val = val.strip().upper()
        if val.endswith('K'):
            total_gdp_copy.loc[country][year] = float(val[:-1]) * 1e3
        elif val.endswith('M'):
            total_gdp_copy.loc[country][year] = float(val[:-1]) * 1e6
        elif val.endswith('B'):
            total_gdp_copy.loc[country][year] = float(val[:-1]) * 1e9
        elif val.endswith('TR'):
            total_gdp_copy.loc[country][year] = float(val[:-2]) * 1e12
        else:
            total_gdp_copy.loc[country][year] = float(val[:-1])

for country in gdp_percap_copy.index:
    for year in gdp_percap_copy:
        val = gdp_percap_copy.loc[country][year]
        if isinstance(val, float):
            continue    
        val = val.strip().upper()
        if val.endswith('K'):
            gdp_percap_copy.loc[country][year] = float(val[:-1]) * 1e3
        elif val.endswith('M'):
            gdp_percap_copy.loc[country][year] = float(val[:-1]) * 1e6
        elif val.endswith('B'):
            gdp_percap_copy.loc[country][year] = float(val[:-1]) * 1e9
        elif val.endswith('TR'):
            gdp_percap_copy.loc[country][year] = float(val[:-2]) * 1e12
        else:
            gdp_percap_copy.loc[country][year] = float(val[:-1])
del country
del year
del val



Raw data loaded in as pandas dataframes. Now need to merge across a new axis called 'Metric'. Outer index is 'Metric', inner index is 'Country'. Use inner join to only include years common to all dataframes.

In [3]:
mask_total_gdp     = total_gdp_copy.index.isin(gdp_percap_copy.index) & total_gdp_copy.index.isin(gini.index)
mask_gdp_percap    = gdp_percap_copy.index.isin(total_gdp_copy.index) & gdp_percap_copy.index.isin(gini.index)
mask_gini          = gini.index.isin(total_gdp_copy.index) & gini.index.isin(gdp_percap_copy.index)

total_gdp_filtered     = total_gdp_copy[mask_total_gdp]
gdp_percap_filtered    = gdp_percap_copy[mask_gdp_percap]
gini_filtered          = gini[mask_gini]

In [5]:
combined_data = pd.concat([total_gdp_filtered, gdp_percap_filtered, gini_filtered], axis=0, keys=['Total GDP', 'GDP Per Capita', 'Gini Coeff'], join='inner')
combined_data.index.names = ['Metric', 'Country']
# new_gini = combined_data.loc['Gini Coeff']
# Canada = combined_data.xs('Canada', level='Country')
# Canada_GDP_2023 = combined_data.xs('Total GDP', level='Metric').loc['Canada', '2023']
# combined_data.shape
# totalGDP_Gini_diff = total_gdp_copy.index.difference(gini.index)
# Gini_totalGDP_diff = gini.index.difference(total_gdp.index)
# GDPpercap_Gini_diff = gdp_percap_copy.index.difference(gini.index)

# all_rejected_countries = list(totalGDP_Gini_diff) + list(Gini_totalGDP_diff)
# print(all_rejected_countries)
del mask_total_gdp
del mask_gdp_percap
del mask_gini

del total_gdp_filtered
del gdp_percap_filtered
del gini_filtered       


In [ ]:
Canada = combined_data.xs('Canada', level='Country')
Canada_GDP_2023 = combined_data.xs('Total GDP', level='Metric').loc['Canada', '2023']

Now, remove rows (countries) that do not meet the threshold for sufficient valid data. For now, lets say we need a minimum of half of the years having a value.

In [ ]:
combined_data_copy = combined_data.copy()
num_all_years = combined_data.shape[1]
threshold_years = num_all_years/2

gdp_data = combined_data_copy.loc['Total GDP']
nan_counts = gdp_data



Tasks going forward:
- User Entry
    - Prompt user to enter some number of countries, and a start year and end year.
    - Returns a tuple of ([list of Strings as country names], [list of start year, end year]).

- Dataframe cleaning:
    Remove countries that have less that length/2 valid numbers.

- Plotting:
    - Plot for each metric containing:
        - all contries and global average for time slice. 

- Presentation:
    - Summary of problem statement
    - Summary of data processing steps taken
    - Figures
    - Formatting into .pptx
